In [0]:
# !pip install tensorflow==2.0.0
# !pip install tensorflow==1.15.0
# !pip install keras==2.3.1
# !pip install tensorflow-gpu
import keras
import keras.backend as K
from keras.layers import Dense, LSTM, concatenate, Input, CuDNNLSTM
from keras.models import Model
from keras.utils import plot_model, to_categorical
from keras.optimizers import Adam, SGD
from keras import metrics
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
# feature_matrix = np.random.normal(size=(1000, 20))
# vocab = ['START', 'C', 'O', '=', '1', '-', 'H', 'END']
# vocab_size = len(vocab)+1

In [0]:
# feature_matrix = []
# for i in range(1000):
#     feature = []
#     feature_values = np.random.normal(size=(20))
#     for j in range(10):
#         feature.append(feature_values)
#     feature = np.array(feature)
#     feature_matrix.append(feature)
# feature_matrix = np.array(feature_matrix).astype(int)
# feature_matrix = feature_matrix.reshape(10000, 20)

In [0]:
feature_matrix = pd.read_csv("descriptors.csv")
y_smiles_text = feature_matrix["molecule_smile"]
max_len = y_smiles_text.apply(len).max() #97
feature_matrix = feature_matrix.drop("molecule_smile", axis=1)
n_features = feature_matrix.shape[1]
X = y_smiles_text.apply(lambda x: '%' + x + '^')
X_padded = X.apply(lambda x: x + '~'*(max_len-len(x)+2)) #99
X = pd.Series(list(X_padded.sum()))
Y_padded = y_smiles_text.apply(lambda x: x + '^' + '~'*(max_len-len(x)+1))
Y = pd.Series(list(Y_padded.sum()))
# print(len(X))
# print(len(Y))
# print(X.shape, Y.shape)
# print(X.apply(len).min(), Y.apply(len).min())
# print(X.apply(len).max(), Y.apply(len).max())

In [7]:
# repeated_feature_matrix = []
# for idx, row in feature_matrix.iterrows():
#   for j in range(99):
#     repeated_feature_matrix.append(row)
# repeated_feature_matrix = np.array(repeated_feature_matrix)
# print(repeated_feature_matrix.shape)

(333927, 90)


In [8]:
# data = pd.concat([X, Y, pd.DataFrame(repeated_feature_matrix)], axis=1)
# data.columns = ['X', 'Y']+list(data.columns[2:])
# data_train, data_test = train_test_split(data, test_size=0.2)
# data_train, data_val = train_test_split(data_train, test_size=0.2)
# X_train = data_train['X']
# Y_train = data_train['Y']
# F_train = data_train.drop(['X', 'Y'], axis=1)
# X_val = data_val['X']
# Y_val = data_val['Y']
# F_val = data_val.drop(['X', 'Y'], axis=1)
# X_test = data_test['X']
# Y_test = data_test['Y']
# F_test = data_test.drop(['X', 'Y'], axis=1)
# print(X_train.shape, Y_train.shape, F_train.shape)
# print(X_val.shape, Y_val.shape, F_val.shape)
# print(X_test.shape, Y_test.shape, F_test.shape)

(213712,) (213712,) (213712, 90)
(53429,) (53429,) (53429, 90)
(66786,) (66786,) (66786, 90)


In [0]:
data = pd.concat([X_padded, Y_padded, feature_matrix], axis=1)
data.columns = ['X', 'Y']+list(data.columns[2:])
data_train, data_test = train_test_split(data, test_size=0.2)
data_train, data_val = train_test_split(data_train, test_size=0.2)
X_train = data_train['X']
Y_train = data_train['Y']
F_train = data_train.drop(['X', 'Y'], axis=1)
X_val = data_val['X']
Y_val = data_val['Y']
F_val = data_val.drop(['X', 'Y'], axis=1)
X_test = data_test['X']
Y_test = data_test['Y']
F_test = data_test.drop(['X', 'Y'], axis=1)

In [24]:
F_W_train = []
for idx, row in F_train.iterrows():
    feature = []
    feature_values = np.array(row)
    for j in range(max_len+2):
        feature.append(feature_values)
    feature = np.array(feature)
    F_W_train.append(feature)
F_W_train = np.array(F_W_train)
F_W_train = F_W_train.reshape(F_W_train.shape[0]*F_W_train.shape[1], F_W_train.shape[2])
print(X_train.shape, Y_train.shape, F_W_train.shape)

(2158,) (2158,) (213642, 90)


In [25]:
F_W_val = []
for idx, row in F_val.iterrows():
    feature = []
    feature_values = np.array(row)
    for j in range(max_len+2):
        feature.append(feature_values)
    feature = np.array(feature)
    F_W_val.append(feature)
F_W_val = np.array(F_W_val)
F_W_val = F_W_val.reshape(F_W_val.shape[0]*F_W_val.shape[1], F_W_val.shape[2])
F_W_val.shape

(53460, 90)

In [26]:
F_W_test = []
for idx, row in F_test.iterrows():
    feature = []
    feature_values = np.array(row)
    for j in range(max_len+2):
        feature.append(feature_values)
    feature = np.array(feature)
    F_W_test.append(feature)
F_W_test = np.array(F_W_test)
F_W_test = F_W_test.reshape(F_W_test.shape[0]*F_W_test.shape[1], F_W_test.shape[2])
F_W_test.shape

(66825, 90)

In [28]:
X_train_str = X_train
X_val_str = X_val
X_test_str = X_test
Y_train_str = Y_train
Y_val_str = Y_val
Y_test_str = Y_test
Y_test_str = Y_test
print(X_train_str.shape, Y_train_str.shape)
print(X_val_str.shape, Y_val_str.shape)
print(X_test_str.shape, Y_test_str.shape)
X_test_str

(2158,) (2158,)
(540,) (540,)
(675,) (675,)


1820    %Cc1c(CC#N)cccc1-c1cccc(NS(=O)(=O)c2ccccc2)c1C...
1746    %O=C(O)c1cc(Cl)c(Br)c(N2CCC(COc3ccc(C(=O)N4CCO...
683     %CCOC(=O)C1(C(=O)Cc2ccccc2)CCCN(C(C)C)C1^~~~~~...
1527    %CSc1ccc(OCCNc2cc(O)ncn2)cc1^~~~~~~~~~~~~~~~~~...
1173    %O=C1CCC[C@@H](C(=O)N(c2c(Cl)cccc2Cl)S(=O)(=O)...
                              ...                        
277     %CN1CCN(c2ccc(C(=O)NCCCOc3c(F)c(F)c(F)c(F)c3F)...
182     %C[C@H](CCC(=O)O)CC(=O)N1CCc2c(c(COCc3ccccc3F)...
443     %CC1(C)CC1Nc1ccc(OCCN2CCCC2)cc1^~~~~~~~~~~~~~~...
2848    %CC(C)(O)c1ccncc1NCc1nc(-c2cccc(Cl)c2)cs1^~~~~...
3101    %O=c1c(S(=O)(=O)N2CCCCCC2)cccn1-c1cccc(Cl)c1C1...
Name: X, Length: 675, dtype: object

In [0]:
# X_train_str = X_train.sum()
# X_val_str = X_val.sum()
# X_test_str = X_test.sum()
# Y_train_str = Y_train.sum()
# Y_val_str = Y_val.sum()
# Y_test_str = Y_test.sum()
# Y_test_str = Y_test.sum()
# print(len(X_train_str), len(Y_train_str))
# print(len(X_val_str), len(Y_val_str))
# print(len(X_test_str), len(Y_test_str))

213642 213642
53460 53460
66825 66825


In [0]:
# X = []
# Y = []
# sample_lengths = np.random.randint(low=5, high=max_sequence_length, size=1000)
# for sample_length in sample_lengths:
#     start_index = 1
#     content = np.random.randint(low=2, high=vocab_size, size=sample_length-1)
#     padding_x = np.zeros(max_sequence_length-sample_length-1)
#     padding_y = np.zeros(max_sequence_length-sample_length)
#     end_index = len(vocab)
#     x = np.append(np.append(np.append(start_index, content), end_index), padding_x)
#     y = np.append(np.append(content, end_index), padding_y)
#     X.append(x)
#     Y.append(y)
# X = np.array(X).astype(int)
# Y = np.array(Y).astype(int)

In [0]:
# X_onehot = []
# Y_onehot = []
# for x, y in zip(X, Y):
#     X_onehot.append(np.eye(9)[x])
#     Y_onehot.append(np.eye(9)[y])
# X_onehot = np.array(X_onehot).astype(int)
# Y_onehot = np.array(Y_onehot).astype(int)

In [0]:
# X_final = X_onehot.reshape(X_onehot.shape[0]*X_onehot.shape[1], X_onehot.shape[2], 1)
# Y_final = Y_onehot.reshape(Y_onehot.shape[0]*Y_onehot.shape[1], Y_onehot.shape[2])

In [0]:
# vocab = list(set(X_train_str+X_val_str+Y_train_str+Y_val_str+X_test_str+Y_test_str))
# vocab_size = len(vocab)
# vocab_size

38

In [16]:
vocab = list(set(list(list(X_train) + list(Y_train) + list(X_val))))
vocab_size = len(vocab)
vocab_size

38

In [0]:
# X_train_idx = [vocab.index(x) for x in X_train_str]
# Y_train_idx = [vocab.index(x) for x in Y_train_str]
# X_val_idx = [vocab.index(x) for x in X_val_str]
# Y_val_idx = [vocab.index(x) for x in Y_val_str]
# X_test_idx = [vocab.index(x) for x in X_test_str]
# Y_test_idx = [vocab.index(x) for x in Y_test_str]

In [29]:
X_train_idx = []
for row in X_train_str:
  row_idx = [vocab.index(x) for x in row]
  X_train_idx.append(row_idx)
X_val_idx = []
for row in X_val_str:
  row_idx = [vocab.index(x) for x in row]
  X_val_idx.append(row_idx)
X_test_idx = []
for row in X_test_str:
  row_idx = [vocab.index(x) for x in row]
  X_test_idx.append(row_idx)
Y_train_idx = []
for row in Y_train_str:
  row_idx = [vocab.index(x) for x in row]
  Y_train_idx.append(row_idx)
Y_val_idx = []
for row in Y_val_str:
  row_idx = [vocab.index(x) for x in row]
  Y_val_idx.append(row_idx)
Y_test_idx = []
for row in Y_test_str:
  row_idx = [vocab.index(x) for x in row]
  Y_test_idx.append(row_idx)
# Y_train_idx = [vocab.index(x) for x in Y_train_str]
# X_val_idx = [vocab.index(x) for x in X_val_str]
# Y_val_idx = [vocab.index(x) for x in Y_val_str]
# X_test_idx = [vocab.index(x) for x in X_test_str]
# Y_test_idx = [vocab.index(x) for x in Y_test_str]
X_train_idx = np.array(X_train_idx)
X_val_idx = np.array(X_val_idx)
X_test_idx = np.array(X_test_idx)
Y_train_idx = np.array(Y_train_idx)
Y_val_idx = np.array(Y_val_idx)
Y_test_idx = np.array(Y_test_idx)
print(X_train_idx.shape, Y_train_idx.shape)
print(X_val_idx.shape, Y_val_idx.shape)
print(X_test_idx.shape, Y_test_idx.shape)

(2158, 99) (2158, 99)
(540, 99) (540, 99)
(675, 99) (675, 99)


In [13]:
X_train

254281    c
250627    ~
190173    ~
254346    N
101303    C
         ..
270504    3
74687     c
156038    2
262121    ~
266710    n
Name: X, Length: 213712, dtype: object

In [19]:
# X_train_idx = []
# for row in X_train:
#   row_idx = vocab.index(row)
#   X_train_idx.append(row_idx)
# X_val_idx = []
# for row in X_val_str:
#   row_idx = [vocab.index(x) for x in row]
#   X_val_idx.append(row_idx)
# X_test_idx = []
# for row in X_test_str:
#   row_idx = [vocab.index(x) for x in row]
#   X_test_idx.append(row_idx)
# Y_train_idx = []
# for row in Y_train_str:
#   row_idx = [vocab.index(x) for x in row]
#   Y_train_idx.append(row_idx)
# Y_val_idx = []
# for row in Y_val_str:
#   row_idx = [vocab.index(x) for x in row]
#   Y_val_idx.append(row_idx)
# Y_test_idx = []
# for row in Y_test_str:
#   row_idx = [vocab.index(x) for x in row]
#   Y_test_idx.append(row_idx)
# # Y_train_idx = [vocab.index(x) for x in Y_train_str]
# # X_val_idx = [vocab.index(x) for x in X_val_str]
# # Y_val_idx = [vocab.index(x) for x in Y_val_str]
# # X_test_idx = [vocab.index(x) for x in X_test_str]
# # Y_test_idx = [vocab.index(x) for x in Y_test_str]
# X_train_idx = np.array(X_train_idx)
# X_val_idx = np.array(X_val_idx)
# X_test_idx = np.array(X_test_idx)
# Y_train_idx = np.array(Y_train_idx)
# Y_val_idx = np.array(Y_val_idx)
# Y_test_idx = np.array(Y_test_idx)
# print(X_train_idx.shape, Y_train_idx.shape)
# print(X_val_idx.shape, Y_val_idx.shape)
# print(X_test_idx.shape, Y_test_idx.shape)
X_train.values

array(['c', '~', '~', ..., '2', '~', 'n'], dtype=object)

In [30]:
X_train_onehot = to_categorical(X_train_idx)
X_val_onehot = to_categorical(X_val_idx)
X_test_onehot = to_categorical(X_test_idx)

Y_train_onehot = to_categorical(Y_train_idx)
Y_val_onehot = to_categorical(Y_val_idx)
Y_test_onehot = to_categorical(Y_test_idx)
print(X_train_onehot.shape, Y_train_onehot.shape)
print(X_val_onehot.shape, Y_val_onehot.shape)
print(X_test_onehot.shape, Y_test_onehot.shape)

(2158, 99, 38) (2158, 99, 38)
(540, 99, 38) (540, 99, 38)
(675, 99, 38) (675, 99, 38)


In [31]:
F_train = np.array(F_train).reshape(F_train.shape[0], 1, F_train.shape[1]).repeat(99, axis=1)
F_val = np.array(F_val).reshape(F_val.shape[0], 1, F_val.shape[1]).repeat(99, axis=1)
F_test = np.array(F_test).reshape(F_test.shape[0], 1, F_test.shape[1]).repeat(99, axis=1)
print(F_train.shape, F_val.shape, F_test.shape)

(2158, 99, 90) (540, 99, 90) (675, 99, 90)


In [0]:
print(X_train_onehot.shape, F_train.shape)

(2158, 99, 38) (2158, 99, 90)


In [32]:
X_F_train = np.concatenate([X_train_onehot, F_train], axis=2)
X_F_val = np.concatenate([X_val_onehot, F_val], axis=2)
X_F_test = np.concatenate([X_test_onehot, F_test], axis=2)
print(X_F_train.shape, Y_train_onehot.shape)
print(X_F_val.shape, Y_val_onehot.shape)
print(X_F_test.shape, Y_test_onehot.shape)

(2158, 99, 128) (2158, 99, 38)
(540, 99, 128) (540, 99, 38)
(675, 99, 128) (675, 99, 38)


In [33]:
encoder_inputs_train = X_F_train
decoder_inputs_train = X_train_onehot
decoder_outputs_train = Y_train_onehot
encoder_inputs_val = X_F_val
decoder_inputs_val = X_val_onehot
decoder_outputs_val = Y_val_onehot
encoder_inputs_test = X_F_test
decoder_inputs_test = X_test_onehot
decoder_outputs_test = Y_test_onehot
print(encoder_inputs_train.shape, decoder_inputs_train.shape, decoder_outputs_train.shape)
print(encoder_inputs_val.shape, decoder_inputs_val.shape, decoder_outputs_val.shape)
print(encoder_inputs_test.shape, decoder_inputs_test.shape, decoder_outputs_test.shape)

(2158, 99, 128) (2158, 99, 38) (2158, 99, 38)
(540, 99, 128) (540, 99, 38) (540, 99, 38)
(675, 99, 128) (675, 99, 38) (675, 99, 38)


In [0]:
# X_F_train_onehot = X_F_train.reshape(X_F_train.shape[0], X_F_train.shape[1], 1)
# X_F_val_onehot = X_F_val.reshape(X_F_val.shape[0], X_F_val.shape[1], 1)
# X_F_test_onehot = X_F_test.reshape(X_F_test.shape[0], X_F_test.shape[1], 1)

In [36]:
encoder_inputs = Input(shape=(None, 128))
encoder = CuDNNLSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, 38))
decoder_lstm = CuDNNLSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(38, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, 38)     0                                            
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        [(None, 64), (None,  49664       input_3[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)        [(None, None, 64), ( 26624       input_4[0][0]                    
                                                                 cu_dnnlstm_1[0][1]         

In [53]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy])
checkpoint = ModelCheckpoint("model", monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)
history = model.fit([encoder_inputs_train, decoder_inputs_train], decoder_outputs_train,
          batch_size=64,
          epochs=100,
          validation_data=([encoder_inputs_val, decoder_inputs_val], decoder_outputs_val),
          callbacks=[checkpoint])

Train on 2158 samples, validate on 540 samples
Epoch 1/100
2158/2158 [==============================] - 2s 845us/step - loss: 0.4898 - categorical_accuracy: 0.8324 - val_loss: 0.4895 - val_categorical_accuracy: 0.8329
Epoch 2/100
2158/2158 [==============================] - 1s 328us/step - loss: 0.4568 - categorical_accuracy: 0.8436 - val_loss: 0.4796 - val_categorical_accuracy: 0.8350
Epoch 3/100
2158/2158 [==============================] - 1s 330us/step - loss: 0.4513 - categorical_accuracy: 0.8455 - val_loss: 0.4774 - val_categorical_accuracy: 0.8372
Epoch 4/100
2158/2158 [==============================] - 1s 325us/step - loss: 0.4490 - categorical_accuracy: 0.8462 - val_loss: 0.4764 - val_categorical_accuracy: 0.8383
Epoch 5/100
2158/2158 [==============================] - 1s 332us/step - loss: 0.4451 - categorical_accuracy: 0.8477 - val_loss: 0.4711 - val_categorical_accuracy: 0.8391

Epoch 00005: val_categorical_accuracy improved from -inf to 0.83913, saving model to model
Epoch 

In [0]:
# # max_sequence_length = 10
# # n_features = 20
# feature_input = Input(shape=(n_features,), name='feature_input')
# # features_dense = Dense(64, activation='relu')(feature_input)

# lstm_input = Input(shape=(vocab_size,1), name='lstm_input')
# lstm_output = CuDNNLSTM(units=16)(lstm_input)
# # lstm_output_1 = CuDNNLSTM(units=64)(lstm_output)
# concat = concatenate([feature_input, lstm_output])
# # dense_concat = Dense(64, activation='relu')(concat)
# model_output = Dense(vocab_size, activation='softmax')(concat)

# model = Model(inputs=[feature_input, lstm_input], outputs=model_output)
# model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_input (InputLayer)         (None, 38, 1)        0                                            
__________________________________________________________________________________________________
feature_input (InputLayer)      (None, 90)           0                                            
__________________________________________________________________________________________________
cu_dnnlstm_19 (CuDNNLSTM)       (None, 16)           1216        lstm_input[0][0]                 
__________________________________________________________________________________________________
concatenate_12 (Concatenate)    (None, 106)          0           feature_input[0][0]              
                                                                 cu_dnnlstm_19[0][0]       

In [0]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# optimizer = SGD(lr=0.001, momentum=0.01, nesterov=True)
checkpoint = ModelCheckpoint("model", monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[metrics.categorical_accuracy])
history = model.fit(x={'lstm_input': X_train_onehot, 'feature_input': F_W_train}, 
          y=Y_train_onehot, 
          validation_data=([F_W_val, X_val_onehot], Y_val_onehot),
          epochs=100, batch_size=128, callbacks=[checkpoint])

In [0]:
starting_char = np.zeros(vocab_size)
starting_char[vocab.index('%')] = 1

In [0]:
predictions = model.predict([F_W_val, X_val_onehot])

In [0]:
prediction_str = ''
for prediction_idx in np.argmax(predictions, axis=1):
  prediction_str += vocab[prediction_idx]
print(prediction_str)

CC)ccccccC)cccCCCcCCCO)c)CCCcCCcC~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~C(ccccCcccccCcc)ccccC)cc)cc3cccccc((CC(CO)cn(@@](cn(@]()((n(@](C()cn(@@](cCc(c~~~~~~~~~~~~~~~~~~~~~CC)c^ccC)ccCCCccCCCcCCCCcCCcCcCCccc^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~C)OCCCCCCccccccccccccccCr)cccccCcCCCC)cCCc~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~CCCCCCCCC)CCO)CCCCH](CCC)CCO)CCCCCCCCCCCCO)C)CCCCCCCCC1C11111C~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~C(()(=O)cccccccc=cccccc=()((=(=O)c)c(()O)cccc(ccccccc(~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~C(cccccC(((CO)ccccccC()cccc()ccc~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~C)O(C(((CO)cCc(((cC((cc(CCcccccC()cccc)c(CO)c)cccccccCr)cccc~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Cl)ccccCr)ccC)cccc(ccccccCll)ccc~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~CCCCCCCCCCCCCO)C)CcCccccC)CCcC)C~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~CCCCCCCCCC

In [0]:
print(Y_val_str)

COc1cccc(F)c1N(C)CC(=O)OC(C)(C)C^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Cn1nc(-c2nc(-c3cccc(F)c3)no2)c2c1CCN(C(=O)[C@@H]1[C@H]3CC[C@H](C3)[C@@H]1N)C2^~~~~~~~~~~~~~~~~~~~~~COc1cc(F)c(CN)c(NC2(C(C)(C)C)CC2)c1^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~O=C(NC(Cc1ccccc1)c1ccc(Br)cc1)C1CCC(O)CC1^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~CC(C)(C)OC(=O)N[C@H](CCOC(=O)CCC1CCCN1C(=O)OC(C)(C)C)c1ccccc1^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~CCOC(=O)c1oc2cc(-c3ccc(N4CC(C(=O)O)CC4=O)cc3C)ccc2c1N^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Nc1nnc(CNC(=O)c2ccc(Cl)cc2Cl)s1^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~O=C(CCC(=O)N1CCC2(CC1)CN(c1ccc(Cl)cc1F)C(=O)O2)c1ccc(Br)cc1^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~COc1cc(Br)c(F)cc1Nc1cccc(CCO)c1^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~CCCCCC(N)CC(=O)OCc1cccc(OC)c1OC^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~CC1CCCN(CC

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 99, 38))
    # Populate the first character of target sequence with the start character.
    target_seq[0, :, 6] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = vocab[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '^' or
           len(decoded_sentence) > 99):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 99, 38))
        target_seq[0, :, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [59]:
F_train[0].shape
start_chars = np.zeros((1, 38))
start_chars[:,vocab.index('%')] = 1
test_input = np.concatenate([start_chars, F_train[20][0][np.newaxis,:]], axis=1).reshape(1, 1, 128)
decode_sequence(test_input)

')1^'

In [0]:
encoder_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 99, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 64), (None, 64),  49408     
Total params: 49,408
Trainable params: 49,408
Non-trainable params: 0
_________________________________________________________________


In [0]:
decoder_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 99, 38)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 99, 64), (No 26368       input_2[0][0]                    
                                                                 input_3[0][0]              